# Contents
1) create a search table with key=term and value paragragh in raw LaTeXML output

2) a term-reference is {term, art_addr, parag_index, p_tag, tfidf}

3) serialize the list of term-references

TODO:
- Some `<para>` tags don't have a `<p>` inside so I just used recutext. *ADD* the missing p tag

In [5]:
from lxml import etree
import pickle
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import glob
from tqdm import tqdm
from sklearn import linear_model
import tarfile
from tqdm import tqdm
from marshmallow import Schema, fields, pprint
from random import random
from dataclasses import dataclass, field
from sklearn.feature_extraction.text import TfidfVectorizer

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, os.path.join(parentdir, 'embed'))
sys.path.insert(0, parentdir)

%load_ext autoreload
%autoreload 2

from clean_and_token_text import normalize_text, normalize_phrase, join_xml_para_and_write, ReadGlossary
import parsing_xml as px
import peep_tar as peep
from enum import Enum

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
general_path = 'math20/2003_003'
argot_path = '/media/hd1/glossary/NN.v1/' + general_path + '.xml.gz'
prom_path = '/media/hd1/promath/' + general_path + '.tar.gz'
join_path = '/media/hd1/cleaned_text/joined_math12-34_04-01/' + general_path + 'xml.gz'
ns = {'latexml': 'http://dlmf.nist.gov/LaTeXML' }

argot = etree.parse(argot_path)
with tarfile.open(prom_path) as tar_fobj:
    art_lst = [k.get_info()['name'] for k in tar_fobj.getmembers()]
    members = tar_fobj.getmembers()

In [14]:
glossary_file_lst = glob.glob('/media/hd1/glossary/NN.v1/math9*/*.xml.gz')
glossary_file_lst += glob.glob('/media/hd1/glossary/NN.v1/math0*/*.xml.gz')

#RG = ReadGlossary('/media/hd1/glossary/v3/math*/*.xml.gz', '/media/hd1/glossary/NN.v1/math*/*')
#vocab = RG.ntc_intersect('relative')

corpus = []
term_ref_lst = []
for glossary_file in tqdm(glossary_file_lst):
    # argot_file format: /media/hd1/glossary/NN.v1/math95/9506_001.xml.gz
    math_year = glossary_file.split('/')[-2]
    subfilename = glossary_file.split('/')[-1].split('.')[0] # should be 9506_001
    promath_file = os.path.join('/media/hd1/promath', math_year, subfilename + '.tar.gz')
    joined_file = os.path.join('/media/hd1/cleaned_text/joined_math12-34_04-01/',
                               math_year, subfilename + '.xml.gz')
    
    glossary_xml = etree.parse(glossary_file)
    joined_xml = etree.parse(joined_file)
    promath_tarfobj = tarfile.open(promath_file)
    for art in glossary_xml.findall('./article'):
        art_name = art.get('name')
        # need to use only the basename of the article
        art_basename = os.path.basename(art_name)
        joined_name_results = joined_xml.xpath('./article[@name="{}"]'.format(art_basename))
        # POPULATE THE CORPUS
        if len(joined_name_results) > 0:
            joined_art_text = ''
            for parag in joined_name_results[0].findall('./parag'):
                text = '' if parag.text is None else parag.text
                joined_art_text += (text + ' ')
            corpus.append(joined_art_text)
        else:
            print(f'joined name search results empty art_name={art_name}')
            corpus.append('')
        corpus_index = len(corpus) - 1
        
        # OPEN THE PROMATH FILE
        try:
            #promath_obj = peep.tar(promath_file, art_name)[1].exml
            promath_obj = px.DefinitionsXML(promath_tarfobj.extractfile(art_name))
        except AttributeError:
            print(f'{art_name} gave attributeError')
        # this list is in sync with glossary paragraph index
        promath_parag_lst = promath_obj.para_list()
        
        # LOOP THROUGH THE DFDUMS
        for defin in art.findall('./definition'):
            p_index = int(defin.get('index'))
            for term_raw in defin.findall('./dfndum'):
                term = normalize_text(term_raw.text)
                if term in vocab:
                    term_ref_lst.append((
                           term,
                            art_name,
                            p_index,
                            etree.tostring(promath_parag_lst[p_index]).decode('utf-8'),
                            corpus_index))
    promath_tarfobj.close()


  1%|          | 20/2816 [00:00<00:14, 186.45it/s]

found [2816, 2816] files


100%|██████████| 342/342 [1:11:45<00:00, 12.59s/it]


In [17]:
# COMPUTE THE TDIDF MATRIX
vocab_ = list(set([t.replace(' ', '_') for t in vocab]))
tvect = TfidfVectorizer(sublinear_tf=True, norm='l1', vocabulary=vocab_)
                
%time ttrans = tvect.fit_transform(corpus)
print(f'The shape of the resulting matrix is: {ttrans.shape}')

CPU times: user 2min 47s, sys: 1.03 s, total: 2min 48s
Wall time: 2min 48s
The shape of the resulting matrix is: (83840, 347047)


In [19]:
def get_tfidf(term_, corpus_index_, vocab_):
    ter = term_.replace(' ', '_')
    tindex = vocab_.index(ter)
    return ttrans[corpus_index_, tindex]

get_tfidf('_inline_math_', 6112, vocab_)

0.00614318996893933

In [42]:
@dataclass
class TermReference(Schema):
    term : str
    addr : str
    index : int
    p_tag : str
    tfidf : float = field(default_factory=random)
    
class TermRefSchema(Schema):
    term = fields.String()
    addr = fields.String()
    index = fields.Int()
    p_tag = fields.String()
    tfidf = fields.Float()

TR_lst=[]
for tr in term_ref_lst[500_000:510_000]:
    TR_lst.append(TermReference(
        term= tr[0],
           addr = tr[1],
           index = tr[2],
          p_tag = tr[3],
           tfidf = get_tfidf(tr[0], tr[4], vocab_)))

# Serialize all the data and save to json file
trs = TermRefSchema(many=True)
with open('/home/luis/rm_me/math_json/'+ 'half_10K' + '.json', 'w') as fobj:
    fobj.write(trs.dumps(TR_lst))

In [187]:
# check if it is possible to find the same paragraph in promath, argot-glossary, joined
def check_sync(parag_index, art_ind_in_argot, general_path_):
    argot_path = '/media/hd1/glossary/NN.v1/' + general_path_ + '.xml.gz'
    prom_path = '/media/hd1/promath/' + general_path_ + '.tar.gz'
    join_path = '/media/hd1/cleaned_text/joined_math12-34_04-01/' +\
                                general_path_ + '.xml.gz'
    
    # first get the paragraph index from argot
    argot_xml = etree.parse(argot_path)
    article_ind = argot_xml.getroot()[art_ind_in_argot]
    argot_def = article_ind[parag_index]
    article_name = article_ind.get('name')
    print(f'------------ ARGOT {article_name} -------------- ')
    print(etree.tostring(argot_def))
    parag_ind = int(argot_def.get('index'))
    
    #for k,tarobj in enumerate(peep.tar_iter(prom_path, '.xml')):
    #    if k == art_ind_in_argot:
    
    #promath_obj = etree.parse(tarobj[1])
    promath_obj = peep.tar(prom_path, article_name)[1].exml
    parag = promath_obj.findall('.//latexml:para', namespaces=ns)[parag_ind]
    print('-------------- PROMath ------------')
    print(px.recutext_xml(parag))
    
    joined_xml = etree.parse(join_path)
    art_basename = article_name.split('/')[-1]
    joined_ind = joined_xml.xpath('./article[@name="{}"]'.format(art_basename))[0]
    joined_para = joined_ind[parag_ind]
    article_name_in_join = joined_ind.get('name')
    print(f'---------------- JOINED {article_name_in_join}---------------')
    print(etree.tostring(joined_para))
            
check_sync(1, 5, 'math95/9505_001')

------------ ARGOT 9505_001/math.9505216/math.9505216.xml -------------- 
b'<definition index="15">\n      <stmnt> Given _inline_math_, _inline_math_ is in _inline_math_ if the following is true. _inline_math_ is a disjoint decomposition. _inline_math_ and _inline_math_ are in _inline_math_ where _inline_math_, _inline_math_. There is _inline_math_, an isomorphism between _inline_math_ and _inline_math_. _inline_math_ is the identity. For _inline_math_ the sets _inline_math_ and _inline_math_ are isomorphic. _inline_math_. _inline_math_ is one-to-one and takes only values outside _inline_math_ (which is the same as _inline_math_). _inline_math_. We make _inline_math_ and the ordering on _inline_math_ is the one generated by this. </stmnt>\n    <dfndum>disjoint decomposition</dfndum><dfndum>isomorphic</dfndum><dfndum>one-to-one</dfndum></definition>\n  '
-------------- PROMath ------------
 Given _inline_math_, _inline_math_ is in _inline_math_ if the following is true. _inline_math_ is

In [16]:
def get_para(art_addr, para, tar_path=prom_path, run_recutext=True):
    # Get a paragraph from an article compressed in a tar file
    with tarfile.open(tar_path) as tar_fobj:
        xml_fobj = tar_fobj.extractfile(art_addr)
        art = etree.parse(xml_fobj)
        para_lst = art.findall('.//latexml:para', namespaces=ns)
        para_tag = para_lst[para]
        #p_tag = etree.tostring(
        #    para_tag.find('latexml:p', namespaces=ns)).decode('utf-8')
        if run_recutext:
            return px.recutext_xml(para_tag)
        else:
            return etree.tostring(p_tag).decode('utf-8')
get_para("1501_005/1501.02441/Tiling.xml", 20)

KeyError: "filename '1501_005/1501.02441/Tiling.xml' not found"

In [17]:
# Define a dictionary Key=term and Value=(article_address, parag_index)
#results = argot.xpath(".//dfndum[text()='isothermic']")
#results = argot.xpath(".//dfndum[contains(text(),'quan')]")
results = argot.xpath(".//dfndum")
term_pair_dict = defaultdict(list)
for r in results:
    parent = r.getparent()
    para_index = int(parent.get('index'))
    grand_parent = parent.getparent()
    gparent_name = grand_parent.get('name')
    term_pair_dict[normalize_phrase(r.text)].append((gparent_name, para_index))
    
term_para_dict = defaultdict(list)
with tarfile.open(prom_path) as tar_fobj:
    for term, pair_lst in tqdm(term_pair_dict.items()):
        for pair in pair_lst:
            xml_fobj = tar_fobj.extractfile(pair[0])
            art = etree.parse(xml_fobj)
            para_lst = art.findall('.//latexml:para', namespaces=ns)
            para_tag = para_lst[pair[1]]
            try:
                #p_tag = etree.tostring(
                #    para_tag.find('.//latexml:p', namespaces=ns)).decode('utf-8').strip()
                p_tag = etree.tostring(
                         para_tag).decode('utf-8').strip()
            except TypeError as e:
                p_tag = px.recutext_xml(para_tag)
                print(e)
            term_para_dict[term].append((*pair, p_tag))
    

100%|██████████| 575/575 [01:08<00:00,  8.37it/s]


In [5]:
# search for term containing some text and get the paragraphs (cleaned)
results = argot.xpath(".//dfndum[contains(text(),'quant')]")
with tarfile.open(prom_path) as tar_fobj:
    for r in results:
        print(etree.tostring(r))
        parent = r.getparent()
        para_index = int(parent.get('index'))
        grand_parent = parent.getparent()
        gparent_name = grand_parent.get('name')
        print(grand_parent.tag, gparent_name, grand_parent.get('num') )
        xml_fobj = tar_fobj.extractfile(gparent_name)
        #art = etree.parse(xml_fobj.extractfile())
        art = etree.parse(xml_fobj)
        para_lst = art.findall('.//latexml:para', namespaces=ns)
        print(px.recutext_xml(para_lst[para_index]))
        print('----------------------------')

b'<dfndum>Locally covariant quantum field</dfndum>'
article 1501_005/1501.02682/1501.02682.xml 131
 Locally covariant quantum field theory _citation_ describes QFT on a category of globally hyperbolic spacetimes _inline_math_. Fixing a spacetime dimension _inline_math_, objects of _inline_math_ are quadruples _inline_math_ where _inline_math_ is a smooth paracompact orientable nonempty _inline_math_-manifold with finitely many connected components, _inline_math_ is a smooth time-orientable metric of signature _inline_math_ on _inline_math_, _inline_math_ and _inline_math_ are choices of orientation and %****␣1501.02682.tex␣Line␣600␣**** time-orientation respectively, 1 The orientation (resp., time-orientation) is conveniently represented as a choice of one of the connected components of the nowhere-zero smooth _inline_math_-forms (resp., _inline_math_-timelike _inline_math_-forms) on _inline_math_. so that the spacetime _inline_math_ is globally hyperbolic. That is, _inline_math_ has n

In [83]:
#join_xml_para_and_write('/media/hd1/promath/math98/9805_001.tar.gz', '/home/luis/rm_me_parsing/out/', lambda s: s)
#join_xml_para_and_write('/media/hd1/promath/math99/9903_001.tar.gz', '/home/luis/rm_me_parsing/out/', lambda s: s)
#join_xml_para_and_write('/media/hd1/promath/math99/9905_001.tar.gz', '/home/luis/rm_me_parsing/out/', lambda s: s)
#join_xml_para_and_write('/media/hd1/promath/math97/9710_001.tar.gz', '/home/luis/rm_me_parsing/out/', lambda s: s)
join_xml_para_and_write('/home/luis/rm_me_parsing/more_problematic_article.tar.gz', '/home/luis/rm_me_parsing/out/', lambda s: s)

File two_problem_articles/math.9805119/math.9805119.xml has double-hyphen in comments, using recover (permisive) parser
The file  two_problem_articles/physics.9710002/physics.9710002.xml  is empty.
The file  two_problem_articles/math.9905174/math.9905174.xml  is empty.
{'good': 1, 'empty': 2, 'bad': 0}


In [76]:
parsero = etree.XMLParser(recover=True, remove_comments=True)
par = etree.parse('/home/luis/rm_me_parsing/9805_001/math.9805119/math.9805119.xml')

XMLSyntaxError: Comment must not contain '--' (double-hyphen), line 250217, column 59 (math.9805119.xml, line 250217)

In [74]:
par.findall('.//latexml:para', namespaces=ns)

[<Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd8337eb440>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd8337eb7c0>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd8337ebd40>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd831044580>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd831044040>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd8329df400>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd8329df080>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd8329df9c0>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd833a82fc0>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd833a82cc0>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd831d83b00>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd836fec680>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd836fec580>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd836fece80>,
 <Element {http://dlmf.nist.gov/LaTeXML}para at 0x7fd836fecc80>,
 <Element {http://dlmf.ni